In [ ]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [ ]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

In [ ]:
import pprint

import torch
import pickle
from PIL import Image
import numpy as np
from pytorch3d.renderer import (
    look_at_view_transform
)
import matplotlib.pyplot as plt
import glob

from pose_est import brute_force_pose_est
import postprocess_dataset
from utils import utils

In [ ]:
pose_est.